In [ ]:
!pip install -qU langchain langchain-community langchain-chroma
!pip install -qU langchain-text-splitters
!pip install -qU sentence-transformers
!pip install -qU transformers accelerate sentencepiece
